In [1]:
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

import pandas as pd
import numpy as np#for text pre-processing
import re, string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')#for model-building
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score# bag of words
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer#for word embedding
import gensim
from gensim.models import Word2Vec

import seaborn as sns
import matplotlib.pyplot as plt

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/verhaarpaf/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/verhaarpaf/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/verhaarpaf/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [13]:
#you can download the data from https://www.kaggle.com/c/nlp-getting-started/data
import os
df_train=pd.read_csv('train.csv')
df_test=pd.read_csv('test.csv')
print(df_train.columns)
print(df_test.columns)

Index(['id', 'keyword', 'location', 'text', 'target'], dtype='object')
Index(['id', 'keyword', 'location', 'text'], dtype='object')


https://github.com/vijayaiitk/NLP-text-classification-model/blob/main/NLP%20text%20classification%20model%20Github.ipynb

In [14]:
print(len(df_train.query('target==0')))
print(len(df_train.query('target==1')))

# print(len(df_test.query('target==0')))
# print(len(df_test.query('target==1')))

4342
3271


There are more tweets with class 0 ( no disaster) than class 1 ( disaster tweets). We can say that the dataset is relatively balanced with 4342 non-disaster tweets (57%) and 3271 disaster tweets (43%). Since the data is balanced, we won’t be applying data-balancing techniques like SMOTE while building the model

In [15]:
df_train.isna().sum()

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

Simple text cleaning processes: Some of the common text cleaning process involves:

* Removing punctuations, special characters, URLs & hashtags
* Removing leading, trailing & extra white spaces/tabs
* Typos, slangs are corrected, abbreviations are written in their long forms
* Stop-word removal: We can remove a list of generic stop words from the English vocabulary using nltk. A few such words are ‘i’,’you’,’a’,’the’,’he’,’which’ etc.

* Stemming: Refers to the process of slicing the end or the beginning of words with the intention of removing affixes(prefix/suffix)

* Lemmatization: It is the process of reducing the word to its base form
* Stemming vs Lemmatization



In [16]:
def preprocess(text):
    text = text.lower() 
    text= text.strip()  
    text=re.compile('<.*?>').sub('', text) 
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)  
    text = re.sub('\s+', ' ', text)  
    text = re.sub(r'\[[0-9]*\]',' ',text) 
    text=re.sub(r'[^\w\s]', '', str(text).lower().strip())
    text = re.sub(r'\d',' ',text) 
    text = re.sub(r'\s+',' ',text) 
    return text


In [17]:
# STOPWORD REMOVAL
def stopword(string):
    a= [i for i in string.split() if i not in stopwords.words('english')]
    return ' '.join(a)#LEMMATIZATION
# Initialize the lemmatizer
wl = WordNetLemmatizer()
 
# This is a helper function to map NTLK position tags
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN# Tokenize the sentence
    
def lemmatizer(string):
    word_pos_tags = nltk.pos_tag(word_tokenize(string)) # Get position tags
    a=[wl.lemmatize(tag[0], get_wordnet_pos(tag[1])) for idx, tag in enumerate(word_pos_tags)] # Map the position tag and lemmatize the word/token
    return " ".join(a)


In [19]:
def finalpreprocess(string):
    return lemmatizer(stopword(preprocess(string)))

df_train['clean_text'] = df_train['text'].apply(finalpreprocess)
df_train.head()


,id,keyword,location,text,target,clean_text
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,deed reason earthquake may allah forgive u
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,forest fire near la ronge sask canada
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,resident ask shelter place notify officer evac...
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,people receive wildfire evacuation order calif...
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,get sent photo ruby alaska smoke wildfires pou...


It’s difficult to work with text data while building Machine learning models since these models need well-defined numerical data. The process to convert text data into numerical data/vector, is called vectorization or in the NLP world, word embedding. Bag-of-Words(BoW) and Word Embedding (with Word2Vec) are two well-known methods for converting text data to numerical data.

There are a few versions of Bag of Words, corresponding to different words scoring methods. We use the Sklearn library to calculate the BoW numerical values using these approaches:

    Count vectors: It builds a vocabulary from a corpus of documents and counts how many times the words appear in each document


Term Frequency-Inverse Document Frequencies (tf-Idf): Count vectors might not be the best representation for converting text data to numerical data. So, instead of simple counting, we can also use an advanced variant of the Bag-of-Words that uses the term frequency–inverse document frequency (or Tf-Idf). Basically, the value of a word increases proportionally to count in the document, but it is inversely proportional to the frequency of the word in the corpus

Word2Vec: One of the major drawbacks of using Bag-of-words techniques is that it can’t capture the meaning or relation of the words from vectors. Word2Vec is one of the most popular technique to learn word embeddings using shallow neural network which is capable of capturing context of a word in a document, semantic and syntactic similarity, relation with other words, etc.

We can use any of these approaches to convert our text data to numerical form which will be used to build the classification model. With this in mind, I am going to first partition the dataset into training set (80%) and test set (20%) using the below-mentioned code


In [22]:
#SPLITTING THE TRAINING DATASET INTO TRAIN AND TEST
X_train, X_test, y_train, y_test = train_test_split(df_train["clean_text"],df_train["target"],test_size=0.2,shuffle=True)#Word2Vec
# Word2Vec runs on tokenized sentences
X_train_tok= [nltk.word_tokenize(i) for i in X_train]  
X_test_tok= [nltk.word_tokenize(i) for i in X_test]

In [ ]:
#Here’s the code for vectorization using Bag-of-Words (with Tf-Idf ) and Word2Vec

#Tf-Idf
tfidf_vectorizer = TfidfVectorizer(use_idf=True)
X_train_vectors_tfidf = tfidf_vectorizer.fit_transform(X_train) 
X_test_vectors_tfidf = tfidf_vectorizer.transform(X_test)

#building Word2Vec model

In [27]:
# create Word2vec model
#here words_f should be a list containing words from each document. say 1st row of the list is words from the 1st document/sentence
#length of words_f is number of documents/sentences in your dataset
df_train['clean_text_tok']=[nltk.word_tokenize(i) for i in df_train['clean_text']] #convert preprocessed sentence to tokenized sentence
model = Word2Vec(df_train['clean_text_tok'],min_count=1)  #min_count=1 means word should be present at least across all documents,
#if min_count=2 means if the word is present less than 2 times across all the documents then we shouldn't consider it


w2v = dict(zip(model.wv.index_to_key, model.wv.vectors))  #combination of word and its vector



#for converting sentence to vectors/numbers from word vectors result by Word2Vec
class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = len(next(iter(word2vec.values())))

    def fit(self, X, y):
        return self

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])

In [28]:
#SPLITTING THE TRAINING DATASET INTO TRAINING AND VALIDATION
 
# Input: "reviewText", "rating" and "time"
# Target: "log_votes"
X_train, X_val, y_train, y_val = train_test_split(df_train["clean_text"],
                                                  df_train["target"],
                                                  test_size=0.2,
                                                  shuffle=True)
X_train_tok= [nltk.word_tokenize(i) for i in X_train]  #for word2vec
X_val_tok= [nltk.word_tokenize(i) for i in X_val]      #for word2vec

#TF-IDF
# Convert x_train to vector since model can only run on numbers and not words- Fit and transform
tfidf_vectorizer = TfidfVectorizer(use_idf=True)
X_train_vectors_tfidf = tfidf_vectorizer.fit_transform(X_train) #tfidf runs on non-tokenized sentences unlike word2vec
# Only transform x_test (not fit and transform)
X_val_vectors_tfidf = tfidf_vectorizer.transform(X_val) #Don't fit() your TfidfVectorizer to your test data: it will 
#change the word-indexes & weights to match test data. Rather, fit on the training data, then use the same train-data-
#fit model on the test data, to reflect the fact you're analyzing the test data only based on what was learned without 
#it, and the have compatible


#Word2vec
# Fit and transform
modelw = MeanEmbeddingVectorizer(w2v)
X_train_vectors_w2v = modelw.transform(X_train_tok)
X_val_vectors_w2v = modelw.transform(X_val_tok)

In [33]:
#FITTING THE CLASSIFICATION MODEL using Logistic Regression(tf-idf)

lr_tfidf=LogisticRegression(solver = 'liblinear', C=10, penalty = 'l2')
lr_tfidf.fit(X_train_vectors_tfidf, y_train)  #model

#Predict y value for test dataset
y_predict = lr_tfidf.predict(X_val_vectors_tfidf)
y_prob = lr_tfidf.predict_proba(X_val_vectors_tfidf)[:,1]
 

print(classification_report(y_val,y_predict))
print('Confusion Matrix:',confusion_matrix(y_val, y_predict))
 
fpr, tpr, thresholds = roc_curve(y_val, y_prob)
roc_auc = auc(fpr, tpr)
print('AUC:', roc_auc) 

              precision    recall  f1-score   support

           0       0.81      0.84      0.82       872
           1       0.77      0.74      0.76       651

    accuracy                           0.80      1523
   macro avg       0.79      0.79      0.79      1523
weighted avg       0.79      0.80      0.79      1523

Confusion Matrix: [[729 143]
 [169 482]]
AUC: 0.8501731633760341


In [34]:
#FITTING THE CLASSIFICATION MODEL using Naive Bayes(tf-idf)
#It's a probabilistic classifier that makes use of Bayes' Theorem, a rule that uses probability to make predictions based on prior knowledge of conditions that might be related. This algorithm is the most suitable for such large dataset as it considers each feature independently, calculates the probability of each category, and then predicts the category with the highest probability.

nb_tfidf = MultinomialNB()
nb_tfidf.fit(X_train_vectors_tfidf, y_train)  #model

#Predict y value for test dataset
y_predict = nb_tfidf.predict(X_val_vectors_tfidf)
y_prob = nb_tfidf.predict_proba(X_val_vectors_tfidf)[:,1]
 

print(classification_report(y_val,y_predict))
print('Confusion Matrix:',confusion_matrix(y_val, y_predict))
 
fpr, tpr, thresholds = roc_curve(y_val, y_prob)
roc_auc = auc(fpr, tpr)
print('AUC:', roc_auc)  


              precision    recall  f1-score   support

           0       0.78      0.89      0.83       872
           1       0.82      0.65      0.73       651

    accuracy                           0.79      1523
   macro avg       0.80      0.77      0.78      1523
weighted avg       0.79      0.79      0.79      1523

Confusion Matrix: [[779  93]
 [226 425]]
AUC: 0.8457762581208866


In [35]:
#FITTING THE CLASSIFICATION MODEL using Logistic Regression (W2v)
lr_w2v=LogisticRegression(solver = 'liblinear', C=10, penalty = 'l2')
lr_w2v.fit(X_train_vectors_w2v, y_train)  #model

#Predict y value for test dataset
y_predict = lr_w2v.predict(X_val_vectors_w2v)
y_prob = lr_w2v.predict_proba(X_val_vectors_w2v)[:,1]
 

print(classification_report(y_val,y_predict))
print('Confusion Matrix:',confusion_matrix(y_val, y_predict))
 
fpr, tpr, thresholds = roc_curve(y_val, y_prob)
roc_auc = auc(fpr, tpr)
print('AUC:', roc_auc)  

              precision    recall  f1-score   support

           0       0.66      0.83      0.74       872
           1       0.66      0.44      0.53       651

    accuracy                           0.66      1523
   macro avg       0.66      0.63      0.63      1523
weighted avg       0.66      0.66      0.65      1523

Confusion Matrix: [[721 151]
 [364 287]]
AUC: 0.7222304429318338


In [37]:
#Testing it on new dataset with the best model
df_test=pd.read_csv('test.csv')  #reading the data
df_test['clean_text'] = df_test['text'].apply(lambda x: finalpreprocess(x)) #preprocess the data
X_test=df_test['clean_text'] 
X_vector=tfidf_vectorizer.transform(X_test) #converting X_test to vector
y_predict = lr_tfidf.predict(X_vector)      #use the trained model on X_vector
y_prob = lr_tfidf.predict_proba(X_vector)[:,1]
df_test['predict_prob']= y_prob
df_test['target']= y_predict
print(df_test.columns)
final=df_test[['id','target']].reset_index(drop=True)
final.to_csv('submission.csv')

Index(['id', 'keyword', 'location', 'text', 'clean_text', 'predict_prob',
       'target'],
      dtype='object')


In [38]:
for i,row in df_test.iterrows():
    print(row['text'], row['target'],row['predict_prob'])

Just happened a terrible car crash 1 0.948446243393988
Heard about #earthquake is different cities, stay safe everyone. 1 0.9083455226834195
there is a forest fire at spot pond, geese are fleeing across the street, I cannot save them all 1 0.9359140946913064
Apocalypse lighting. #Spokane #wildfires 1 0.5266328674917496
Typhoon Soudelor kills 28 in China and Taiwan 1 0.9978916367465076
We're shaking...It's an earthquake 1 0.957953619502436
They'd probably still show more life than Arsenal did yesterday, eh? EH? 0 0.11457567312778774
Hey! How are you? 0 0.12744556809590865
What a nice hat? 0 0.028987424086576445
Fuck off! 0 0.030165268071249793
No I don't like cold! 0 0.057675265320462674
NOOOOOOOOO! Don't do that! 0 0.18875842439389842
No don't tell me that! 0 0.17708077695820826
What if?! 0 0.18875842439389842
Awesome! 0 0.04392056718490509
Birmingham Wholesale Market is ablaze BBC News - Fire breaks out at Birmingham's Wholesale Market http://t.co/irWqCEZWEU 1 0.6964704747670081
@sunk

We all knew Cain wasn't going to die... It's either Val or someone is gonna collapse later on... #Emmerdale #SummerFate 0 0.451977227877727
Interview on The Collapse of Materialism Best #TalkRadio Listen Live: http://t.co/Qn8vTXZJzB 1 0.6193962903680212
West Side water main break causes street collapse (VIDEO) - El Paso Times http://t.co/uP9TWymJlV 1 0.9047081610021737
@fouseyTUBE @zaynmalik I would collapse 0 0.25719555409132805
The Guardian Greece's tax revenues collapse as debt crisis continues The Guardian FreshÛ_ http://t.co/L0ARQL2q2O 1 0.9325175043643312
If Oikawa was in Karasuno I guess I'd just collapse from such an overwhelming happiness. 1 0.5265568327095642
Residents set up sign as tribute to families who lost their lives in the Thane building collapse: The unfortun...  http://t.co/zT6wANhXuj 1 0.9910656207024144
Technical Collapse -&gt;  http://t.co/BfJB5H4tuW     $WLB Down -3.43% to New Low http://t.co/cSXEKREDUg 1 0.6376622467711508
.@ArneJungjohann #Energiewende is dir

Heat wave adding to the misery of internally-displaced Gazans http://t.co/jW3hN9ewFT via @PressTV http://t.co/NYWrkRQ7Kn 1 0.8279200215015955
.POTUS #StrategicPatience is a strategy for #Genocide; refugees; IDP Internally displaced people; horror; etc. https://t.co/rqWuoy1fm4 1 0.7679557852894359
.POTUS #StrategicPatience is a strategy for #Genocide; refugees; IDP Internally displaced people; horror; etc. https://t.co/rqWuoy1fm4 1 0.7679557852894359
Next up: a charity program for disadvantaged people displaced by the tech boom in SF based on Tinder technology.  https://t.co/yNVz1WrgtN 0 0.21735333942625615
#mightyworld Thousands displaced after monsoon rains in Myanmar: Tens of thousands of people have been displac... http://t.co/HkSDk89c6C 1 0.8019923562439983
Angry Woman Openly Accuses NEMA Of Stealing Relief Materials Meant For IDPs: An angry Internally Displaced woman... http://t.co/C1vzJOr2qz 1 0.6770219682898002
Oof. Vikings took an island off the mainland. The noble house who ru

Found out a hurricane is brewing for thursday lol ?????? 0 0.4769592040589349
@ReinhardBonnke is preaching on the Holy Spirit! Holy Spirit blows atmosphere to pieces! Atmosphere is air in a jar. The HS is a hurricane! 0 0.2758339472716033
Would you blow me kisses
If I kept my distance?
Would you send a hurricane
As proof of your existence? 0 0.46035179401616355
HURRICANE GUILLERMO LIVE NOAA TRACKING / LOOPING WED.AUG.5TH ~ http://t.co/QktRt2J1o8 ~  http://t.co/b0JxS5QKkh http://t.co/KP05TOwSfM 1 0.9353216567074519
@leeranaldo expressing his rage about Hurricane Sandy #LCOutOfDoors http://t.co/IwPR2qGYsI 1 0.9309158972377493
@BPGVII_GOAT thanks bro ???? 0 0.08846418329602877
Happy birthday bro ?? @Hurricane_Dolce 0 0.2030784821574839
Brooklyn Sites Get $2.6 Million to Undo Hurricane SandyÛªs Toll http://t.co/HukwD0RE2a 1 0.9232216734102004
I'm a hurricane. 1 0.9290777762604229
Udhampur terror attack: Militants attack police post 2 SPOs injured: Suspected militants tonight attacked a p.

@simplyysacred dude like he screams his soul out 0 0.033943006060689465
@sugayiffer SCREAMS UR SUCH A CUTE HANAYO 0 0.02676657001148932
@ShojoShit /SCREAMS 0 0.04140741415841064
Curvaceous LADY screams like a mad dog in a Nairobi Club See what happened (VIDEO) - #KOT Join Mature... http://t.co/1szi9X22cr 0 0.08203277596315868
*aggressively screams* https://t.co/8bHaejsUUt 0 0.34419121660592156
Oh no!  @ATPWorldTour is letting one of their players scream!  Turning @CitiOpen off. Can't listen to the screams 0 0.018958205069626706
@melodores @Hozier *SCREAMS* 0 0.04140741415841064
SCREAMS I LOVE THIS VINE https://t.co/PKnBZLHOSc 0 0.02807654263970609
OMGGGG SCREAMS I'M THINKING I'LL SEE ACTUAL DICKS 0 0.027236902699072998
The screams when Niall touches his dick yes http://t.co/U5Iu6pfNh6 0 0.06953523226193095
@elephino_ @Thrusta1 no but it's a convenient and seismic shift in 6 months from trying to bury the event and now allegedly 'enhance it' 0 0.38619394724542283
SEISMIC AUDIO SA-15T SA

Wreckage 'conclusively confirmed' as from MH370: Malaysia PM http://t.co/Se1QkNPltS | https://t.co/DnhRviV1dJ 1 0.949478010819806
Wreckage 'Conclusively Confirmed' as From MH370: Malaysia PM: Investigators and the families of those who were... http://t.co/O8sTW4YN8R 1 0.988466106638256
#science Now that a piece of wreckage from flight MH370 has been confirmed on RÌ©union Island is it possible t...  http://t.co/cKQ7tOE82u 1 0.9911249084707369
Wreckage 'Conclusively Confirmed' as From MH370: Malaysia PM 1 0.9838909456530764
Wreckage 'Conclusively Confirmed' as From MH370: Malaysia PM: Investigators and the families of those who were... http://t.co/1RzskdRrDk 1 0.9968705225119228
TOP STORY: wreckage from #MH370 officially confirmed. @janisctv reports on the critical clue from a tiny island. http://t.co/hlWw2P5k9o 1 0.9019222533456877
Wreckage 'Conclusively Confirmed' as From MH370: Malaysia PM http://t.co/MN130C4e2D via @ndtv 1 0.9753203711967808
Wreckage 'Conclusively Confirmed' as From 